<a href="https://colab.research.google.com/github/fintechsteve/modeling-volatility/blob/master/Part_04b_Creating_Turbulence_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 04 (b): Creating a Turbulence Model

### In this section you will:


*   Create a turbulence function that estimates turbulence with a rolling window.
*   Transform a turbulence signal into set of dynamic weights. Track performance of the model.
*   Optimize the lookback and smoothing window.


## Import all necessary libraries

For this piece, we will need the following packages to be available to our environment:

*   Numpy and Pandas (For data manipulation)
*   DateTime (For basic date manipulation)
*   Matplotlib (For timeseries vizualization)

If the packages are not available, install the with "pip install X"

In [ ]:
import numpy as np, pandas as pd
from datetime import datetime, timedelta
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import bisect
from scipy import stats
import seaborn as sns

### Read in data from previously stored returns.pkl file



In [ ]:
with open('./returns.pkl', 'rb') as f:
    returns = pickle.load(f)
    f.close()
returns.head(10)

### Create a function that calculates turbulence for the whole window supplied using the last row as the observed return

Note that for faster computation we can replace ``` np.linalg.pinv(covmat)@(r - mu) ``` with ``` np.linalg.lstsq(covmat, r - mu)[0] ```

Also note that a more proper calculation of turbulence would only use rows 1:n-1 to calculate mean and covariance. But everybody seems to include n, so we aren't going to get upset here because it is simpler.

In [ ]:
def simple_turbulence(rets):
  meanret = rets.mean()
  covrets = rets.cov()
  turbvalue = pd.DataFrame((rets[-1:]-meanret)@np.linalg.lstsq(covrets,(rets[-1:]-meanret).T, rcond=None)[0],index=[rets.index[-1]])
  return turbvalue

In [ ]:
%time
turb_singleday = simple_turbulence(returns)
turb_singleday

### Create a function that calculates rolling turbulence given a lookback window and a smoothing window

To do this, we are going to create a function ```roll``` that generates a rolling window of returns for a specified window length, and a function that applies the turbulence calculation to that window.

In [ ]:
def roll(df, w):
    for i in range(df.shape[0] - w + 1):
        yield pd.DataFrame(df.values[i:i+w, :], df.index[i:i+w], df.columns)
  
def rolling_turbulence(rets, lookback=260, smoothing=20):
    roll_turb = pd.concat([simple_turbulence(retwindow) for retwindow in roll(rets, lookback)], axis=0)
    roll_turb.columns = ['daily']
    roll_turb_smooth = pd.DataFrame(roll_turb.rolling(smoothing).mean()).rename(columns={'daily':'smoothed'})
    rets = rets.join(roll_turb)
    rets = rets.join(roll_turb_smooth)
    return rets[['daily','smoothed']]

In [ ]:
%%time
roll_turb = rolling_turbulence(returns, lookback=260, smoothing=20)
roll_turb.tail(20)

In [ ]:
roll_turb.plot()

### Convert rolling turbulence into a weight (And do so in a function)

Turbulence as a raw statistic ranges between 0 and infinity. It cannot be directly used for investment strategies without some kind of transformation into model weights.

Our most obvious optiions are to create a binary cutoff or linear ranking. We are going to use linear ranking, though there are reasons why you might want to use a binary cutoff (simple on/off model).

The linear ranking, when used in-sample, should map results to a uniform distribution.

In [ ]:
roll_turb['smoothed'].hist(bins=50)

In [ ]:
historical_turb_rank = roll_turb['smoothed'].dropna().rank()/roll_turb['smoothed'].count()
historical_turb_rank.hist(bins=50)

In [ ]:
insample_start_date = '1975-01-02'
insample_end_date = '2000-01-01'
insample_turb = (roll_turb['smoothed'].loc[insample_start_date:insample_end_date]).dropna().values
insample_turb.sort()
num_insample = np.size(insample_turb)
roll_turb['ranked'] = 1-np.true_divide(list(map(lambda x:float('NaN') if np.isnan(x) else bisect.bisect_left(insample_turb,x), roll_turb['smoothed'])),num_insample)
roll_turb['ranked'].plot()

In [ ]:
def turbulence_weights(roll_turb, insample_start_date, insample_end_date):
  insample_turb = (roll_turb['smoothed'].loc[insample_start_date:insample_end_date]).dropna().values
  insample_turb.sort()
  num_insample = np.size(insample_turb)
  turb_weights = pd.DataFrame(1-np.true_divide(list(map(lambda x:float('NaN') if np.isnan(x) else bisect.bisect_left(insample_turb,x), roll_turb['smoothed'])),num_insample), index=roll_turb.index)
  turb_weights.columns=['weights']
  return turb_weights

In [ ]:
turb_weights = turbulence_weights(roll_turb, '1975-01-02', '2000-01-01')
turb_weights.tail(20)

### Quick sanity checks

1) Are the weights aligned with decreasing turbulence?
2) Does the weight inversely correlate with subsequent volatility?

In [ ]:
pd.concat([roll_turb, turb_weights],axis=1).plot(kind='scatter', x='weights', y='smoothed',
                                           color='Red', label='')

In [ ]:
dxy_weight = [0, 0.119, 0.036, 0, 0.136, 0.576, 0, 0, 0.091]
dxy = pd.DataFrame(returns.dot(dxy_weight))
dxy_forward = dxy.shift(-1)
dxy_forward_sq = (dxy_forward**2).rename(columns={0:'dxyfsq'})

In [ ]:
highturbulence = turb_weights['weights']<.25
pd.DataFrame([[np.sqrt(261)*dxy[highturbulence].std(), np.sqrt(261)*dxy[~highturbulence].std()], [sum(highturbulence), sum(~highturbulence)]], index=['std','N'], columns=['High Turb','Low Turb'])

### Calculate the performance of the turbulence model

Leveraging the framework we developed in Part 3, we calculate the excess performance of the model.


In [ ]:
def model_perf(model_wts, dxyfsq, framework_params):
  # Note, since chapter 3, modified the code a little to handle the possibility that column headings are not constant
  # Also added an estimation window, which we will use later
    
  model_vol = pd.DataFrame()
  model_vol['total'] = model_wts[model_wts.columns[0]]*dxyfsq[dxyfsq.columns[0]] 
  model_vol['avg'] = (np.mean(model_wts.loc[framework_params['insample_start_date']:framework_params['insample_end_date']].dropna()[model_wts.columns[0]])*dxyfsq)

  is_model_vol = model_vol.loc[framework_params['insample_start_date']:framework_params['insample_end_date']].dropna() 
  is_vol_rmse = np.sqrt(np.mean(is_model_vol))

  out_model_vol = model_vol.loc[framework_params['outofsample_start_date']:framework_params['outofsample_end_date']].dropna()
  out_vol_rmse = np.sqrt(np.mean(out_model_vol))
  
  model_perf_stats = {'insample rmse': is_vol_rmse['total'], 'insample excess rmse': is_vol_rmse['total'] - is_vol_rmse['avg'],
                        'out-of-sample rmse': out_vol_rmse['total'], 'out-of-sample excess rmse': out_vol_rmse['total'] - out_vol_rmse['avg']}
    
  if framework_params.get('estimationsample_start_date'):
    est_model_vol = model_vol.loc[framework_params['estimationsample_start_date']:framework_params['estimationsample_end_date']].dropna()
    est_vol_rmse = np.sqrt(np.mean(est_model_vol))
    model_perf_stats['estimation rmse'] = est_vol_rmse['total']
    model_perf_stats['estimation excess rmse'] = est_vol_rmse['total'] - est_vol_rmse['avg']
        
  if framework_params.get('verbose',False):
    model_perf_stats['insample model volatility'] = is_model_vol
    model_perf_stats['out-of-sample model volatility'] = out_model_vol
    
  return model_perf_stats

In [ ]:
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01',
                    'outofsample_start_date': '2000-01-02',
                    'outofsample_end_date': '2017-12-26',
                    'verbose': False}

model_perf(turb_weights, dxy_forward_sq, framework_params)

## Test significance of excess volatility

A simple test of the ability of a turbulence model to outperform a random model is to calculate the significance of the average excess volatility.

We want this statistic to be negative, because a good model should reduce volatility.

We can calculate this by looking at the difference between the volatility for the model and the volatility of a model that assigns an average weight to volatility and calculating a t-statistic:

In [ ]:
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01',
                    'outofsample_start_date': '2000-01-02',
                    'outofsample_end_date': '2017-12-26',
                    'verbose': True}

model_perf_stats = model_perf(turb_weights, dxy_forward_sq, framework_params)

stats.ttest_rel(model_perf_stats['insample model volatility']['total'], model_perf_stats['insample model volatility']['avg'])


In [ ]:
stats.ttest_rel(model_perf_stats['out-of-sample model volatility']['total'], model_perf_stats['out-of-sample model volatility']['avg'])

### Grid search for the optimal parameters

We chose a smoothing window of 20 days and a lookback of 260 days. How do we pick the correct parameters?

We need to be careful about holdback periods with this kind of analysis. Let's start by being careful with our sample windows.

### Splitting the sample for hyperparameter tuning

We will limit ourselves to the following sample windows:

insample_start_date: '1975-01-02'
insample_end_date: '1994-12-31'
estimationsample_start_date: '1995-01-01'
estimationsample_end_date: '2004-12-31'
outofsample_start_date: '2005-01-01'
outofsample_end_date: '2017-12-26'

In [ ]:
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '1994-12-31',
                    'estimationsample_start_date': '1995-01-01',
                    'estimationsample_end_date': '2004-12-31',
                    'outofsample_start_date': '2005-01-01',
                    'outofsample_end_date': '2017-12-26',
                    'verbose': False}

def turbulence_models_performance(rets, framework_params, lookback, smoothing):
  roll_turb_signal = rolling_turbulence(rets, lookback=lookback, smoothing=smoothing)
  turb_wts = turbulence_weights(roll_turb_signal, framework_params['insample_start_date'], framework_params['insample_end_date'])
  model_perf_stats = model_perf(turb_wts, dxy_forward_sq, framework_params)
  return {'model_perf_stats':model_perf_stats, 'turb_wts':turb_wts}

some_model_perf_stats = turbulence_models_performance(returns, framework_params, 260, 20)
some_model_perf_stats['model_perf_stats']

We will run for the following lookback windows:
65 days (1 Quarter), 130 days (Half year), 260 days (1 year), 390 days (1.5 years), 520 days (2 years)

and the following smoothing windows:
1 day (daily), 5 days (weekly), 10 days (fortnightly), 20 days (monthishly), 40 days (bimonthly)

**Note this takes a little time to run for all 25 models**

In [ ]:
%%time
lookbacks = [65,130,260,390,520]
smoothings = [1, 5, 10, 20, 40]

index = pd.MultiIndex.from_product([lookbacks,smoothings],names=['lookbacks','smoothings'])

turb_model_perf_stats = pd.DataFrame(index = index, columns = some_model_perf_stats['model_perf_stats'].keys())
turb_model_weights = pd.DataFrame(index = returns.index, columns = index)

for lookback in lookbacks:
  for smoothing in smoothings:
    model_output = turbulence_models_performance(returns, framework_params, lookback=lookback, smoothing=smoothing)
    turb_model_perf_stats.loc[(lookback,smoothing)] = model_output['model_perf_stats']
    turb_model_weights.loc[:,(lookback,smoothing)] = model_output['turb_wts']['weights']

turb_model_perf_stats

In [ ]:
turb_model_weights.tail(20)

In [ ]:
turb_model_weights.to_pickle('./turb_model_weights.pkl')
turb_model_perf_stats.to_pickle('./turb_model_perf_stats.pkl')

### Evaluating the out of sample performance of the optimal model

We plan on taking the model with the lowest estimation period excess RMSE as the optimal parameterization.

Let's look at how well this aligns during the out of sample period.

In [ ]:
fig, ax = plt.subplots()
turb_model_perf_stats.plot(x='estimation excess rmse', y='out-of-sample excess rmse', ax=ax, style='o')

for k, v in turb_model_perf_stats.iterrows():
  ax.annotate(k, (v['estimation excess rmse'],v['out-of-sample excess rmse']))

There is a strong (but by no means 1:1) relationship between performance in the estimation window and out of sample, suggesting that these models exhibit persistent performance associated with paramater choices.

### Exploring a bad idea in a little more detail...

Suppose instead we had used the insample performance to choose the best model.

In [ ]:
fig, ax = plt.subplots()
turb_model_perf_stats.plot(x='insample excess rmse', y='out-of-sample excess rmse', ax=ax, style='o')

for k, v in turb_model_perf_stats.iterrows():
  ax.annotate(k, (v['insample excess rmse'],v['out-of-sample excess rmse']))

While there is a positive relationship between the two, the relationship is weaker. This is because we are overfitting in sample.

**In sample performance should never be used to determine the correct parameterization of a model.**

### Sanity checking with random returns

Before you get too far with any model, it is worthwhile testing how your model performs if the inputs are randomized. This is a good way of identifying information leakage inside the model code.

To do this with turbulence, we are going to simulate 1000 random samples of returns, calculate turbulence on these simulated returns and look at the relationship to subsequent simulated DXY. If we use IID normal returns, there should be no predictive power out of sample.

** Note: This step takes a while to run, depending on computational power **

In [ ]:
%%time
framework_params = {'insample_start_date': '1975-01-02',
                    'insample_end_date': '2000-01-01',
                    'outofsample_start_date': '2000-01-02',
                    'outofsample_end_date': '2017-12-26',
                    'verbose': False}

def random_turbulence_models_by_seed(rets,framework_params, seed):
  np.random.seed(seed=seed)
  randreturns = pd.DataFrame(np.random.randn(rets['AUD'].count(),9), index=rets.index, columns=rets.columns)
  roll_turb_signal = rolling_turbulence(randreturns, lookback=260, smoothing=20)
  turb_wts = turbulence_weights(roll_turb_signal, framework_params['insample_start_date'], framework_params['insample_end_date'])
  model_perf_stats = model_perf(turb_wts, dxy_forward_sq, framework_params)
  return model_perf_stats

# random_turbulence_models_by_seed(returns,framework_params, 1)

rand_turb_perf = pd.DataFrame(list(map(lambda x:random_turbulence_models_by_seed(returns, framework_params, x), range(0,1000))))

In [ ]:
rand_turb_perf.describe()

In [ ]:
rand_turb_perf.plot(x='insample excess rmse', y='out-of-sample excess rmse', style='o')

There is no discernable pattern of performance either in- or out-of-sample.

In [ ]:
rand_turb_perf.to_pickle('./rand_turb_perf.pkl')